# BEE 4750 Homework 5: Solid Waste Disposal

**Name**: Christine Swanson

**ID**: cms549

> **Due Date**
>
> Friday, 11/10/23, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   In Problem 2, you will add a CO<sub>2</sub> constraint to the
    capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [31]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\chris\Box\classwork\2023_Fall\BEE5750\hw\hw05-christinemswanson`


In [32]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Background

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided in the table below.

|    **Disposal Facility**     | **Capacity** (Mg/d) | **Fixed cost** (\$/d) | **Tipping Fee** (\$/Mg) | **Recycling Cost** (\$/Mg) |
|:-------------:|:-------------:|:-------------:|:----------:|:--------------:|
|           Landfill           |         200         |         2000          |            50            |                            |
| Materials Recycling Facility |         350         |         1500          |            7             |    40 (per Mg recycled)    |
|   Waste-to-Energy Facility   |         210         |         2500          |            60            |                            |

Transportation costs are \$1.5/Mg-km, and the relative distances between
the cities and facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

|   **Component**   | **% of total mass** | **Combustion ash** (%) | **MRF Recycling rate** (%) |
|:---------------------:|:--------------:|:---------------:|:---------------:|
|    Food Wastes    |         15          |           8            |             0              |
| Paper & Cardboard |         40          |           7            |             55             |
|     Plastics      |          5          |           5            |             15             |
|     Textiles      |          3          |           10           |             10             |
|  Rubber, Leather  |          2          |           15           |             0              |
|       Wood        |          5          |           2            |             30             |
|    Yard Wastes    |         18          |           2            |             40             |
|       Glass       |          4          |          100           |             60             |
|      Ferrous      |          2          |          100           |             75             |
|     Aluminum      |          2          |          100           |             80             |
|    Other Metal    |          1          |          100           |             50             |
|   Miscellaneous   |          3          |           70           |             0              |

The information in the above table will help you determine the overall
recycling and ash fractions. Note that the recycling residuals, which
may be sent to either landfill or the WTE, have different ash content
than the ash content of the original MSW. You will need to determine
these fractions to construct your mass balance constraints.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

## Problems (Total: 40 Points)

### Problem 1 (22 points)

In this problem, you will develop an optimal disposal plan for the two
cities.

#### Problem 1.1 (3 points)

Based on the information above, calculate the overall recycling and ash
fractions for the waste produced by each city.

**Overall recycling fraction:**

Recycling fraction = (0.55 paper and cardboard * 0.4) + (0.15 plastics * 0.05) + (0.10 textiles * 0.03) + (0.30 wood * 0.05) + (0.40 yard waste * 0.18) + (0.60 glass * 0.04) + (0.75 ferrous * 0.02) + (0.80 aluminum * 0.02) + (0.50 other metal * 0.01)

Recycling fraction = (0.22) + (0.0075) + (0.003) + (0.015) + (0.072) + (0.024) + (0.015) + (0.016) + (0.005)

Recycling fraction = 0.3775 = **38%**

**Overall ash fraction:**

Ash fraction = (0.08 food wastes * 0.15) + (0.07 paper and cardboard * 0.40) + (0.05 plastics * 0.05) + (0.10 textiles * 0.03) + (0.15 rubber and leather * 0.02) + (0.02 wood * 0.05) + (0.02 yard wastes * 0.18) + (1 glass * 0.04) + (1 ferrous * 0.02) + (1 aluminum * 0.02) + (1 other metal * 0.01) + (0.70 miscellaneous * 0.03)

Ash fraction = (0.012) + (0.028) + (0.0025) + (0.003) + (0.003) + (0.001) + (0.0036) + (0.04) + (0.02) + (0.02) + (0.01) + (0.021)

Ash fraction = 0.1641 = **16%**

#### Problem 1.2 (2 points)

What are the decision variables for your optimization problem? Provide
notation and variable meaning.

**Decision variables:**

$W_{i,j}$: Waste transported from city i to disposal j. Units in Mg/day

$R_{k,j}$: Residual waste transported from disposal k to disposal j. Units in Mg/day. 

$Y_{j}$: Operational status (on/off) of disposal j. Binary variable; unitless. 

#### Problem 1.3 (3 points)

Formulate the objective function. Make sure to include any needed
derivations or justifications for your equation(s).

**Response:** Our goal in this problem is to minimize our total costs. We need to consider transportation and disposal costs; the disposal costs include both fixed and varaible costs. To derive the transportation costs, we can consider: 

$a_{i,j}$: Cost of transporting waste form source ${i}$ to disposal ${j}$. Units in $/Mg-km. 

$l_{i,j}$: Distance between source ${i}$ and disposal ${j}$. Units in km. 

Putting this all together, we get: 

$$\begin{equation}
\begin{aligned}
& \text{Transportation Costs} = \displaystyle\sum_{i∈I, j∈J, k∈K} (a_{i,j})(l_{i,j})(W_{i,j})(R_{k,j})
\\

\end{aligned} 
\end{equation}$$

To derive the disposal costs, we can consider: 

$c_{j}$: Fixed costs of operating disposal ${j}$. Units in $/day.

$b_{j}$: Variable cost of disposing waste at disposal ${j}$. Units $/Mg. 

Putting this all together, we get: 

$$\begin{equation}
\begin{aligned}
& \text{Disposal Costs} = \displaystyle\sum_{j∈J} [c_{j} + b_{j} + \displaystyle\sum_{i∈I, k∈K} (W_{i,j})(R_{k,j})]
\\

\end{aligned} 
\end{equation}$$

**So, the objective function is:**

$$\begin{equation}
\begin{aligned}
& \min_{W_{i,j}, Y_{j}, R_{k,j}} & Z = \displaystyle\sum_{i∈I, j∈J, k∈K} (a_{i,j})(l_{i,j})(W_{i,j})(R_{k,j}) +
\displaystyle\sum_{j∈J}[(c_{j})(Y_{j}) + \displaystyle\sum_{i∈I, k∈K} (b_{j})(W_{i,j})(R_{k,j})]
\\

\end{aligned} 
\end{equation}$$

**But, we can simplify the objective function by combining the transportation and disposal costs. Let LF = 1, MRF = 2, and WTE = 3 for the subscripts of the variables below. We obtain:**

$$\begin{equation}
\begin{aligned}
\min_{W,R,Y} & Z = 57.5W_{11} + 67.2W_{12} + 82.5W_{13} + 72.5W_{21} + 59.7W_{22} + 75W_{23} + 69.5W_{31} + 89.7W_{32} + 90W_{33} + 98R_{21} + 
\\ &
77R_{31} + 82.5R_{23} + 2500Y_{3} + 1500Y_{2} + 2000Y_{1}
\\

\end{aligned} 
\end{equation}$$

#### Problem 1.4 (4 points)

Derive all relevant constraints. Make sure to include any needed
justifications or derivations.

**Constraints:**

**Mass balance constraints (city):**

*City 1:* $W_{11}$ + $W_{12}$ + $W_{13}$ = 100 (Mg/day)

*City 2:* $W_{21}$ + $W_{22}$ + $W_{23}$ = 90 (Mg/day)

*City 3:* $W_{31}$ + $W_{32}$ + $W_{33}$ = 120 (Mg/day)

**Mass balance constraints (residuals):**

$R_{23}$ + $R_{21}$ = $0.62$($W_{12}$ + $W_{22}$ + $W_{32}$) (MRF)

$R_{31}$ = $0.16$($W_{13}$ + $W_{23}$ + $W_{33}$ + $R_{23}$) (WTE)

**Disposal limit constraints:**

WTE: $W_{13}$ + $W_{23}$ + $W_{33}$ + $R_{23}$ $\leq$ $210$ (Mg/day)

MRF: $W_{12}$ + $W_{22}$ + $W_{32}$ $\leq$ $350$ (Mg/day)

LF: $W_{11}$ + $W_{21}$ + $W_{31}$ + $R_{21}$ + $R_{31}$ $\leq$ $200$ (Mg/day)

**Commitment and Non-negativity constraints:**

$Y_{1} = 1$ (landfill always on)

$Y_{2}$ = {$0$ if $W_{12}$ + $W_{22}$ + $W_{32}$ = $0$; 1 else} (MRF)

$Y_{3}$ = {$0$ if $W_{13}$ + $W_{23}$ + $W_{33}$ + $R_{23}$ = $0$; 1 else} (WTE)

$W_{i,j}$,$R_{k,j}$ $\geq$ $0$

Note for the residual mass balance constraints, I needed to include the overall recycling and ash fractions that I found in problem 1.1. I was able to devise these constraints based off of my diagram for the system, which is shown in problem 1.7. 

#### Problem 1.5 (3 points)

Implement your optimization problem in `JuMP`.

In [40]:
# Step 1: Initialize model with solver
waste_model = Model(HiGHS.Optimizer)

# Step 2.1: Define relevant variables
I = 1:3 # 3 cities
J = 1:3 # 3 disposals
K = 1:3 # 3 disposals for residuals 

# Coefficients for the objective function
c_W = [57.5 67.2 82.5; 72.5 59.7 75.0; 69.5 89.7 90.0]
c_R = [0.0 98.0 0.0; 0.0 0.0 77.0; 0.0 0.0 82.5]
fixed_costs = [2000, 1500, 2500]

# Step 2.2: Define decision variables
@variable(waste_model, W[i in I, j in J] >= 0) # waste transported from city i to disposal j
@variable(waste_model, R[k in K, j in J] >= 0) # residual waste transported b/w disposals
@variable(waste_model, Y[j in J], Bin) # operational status (binary)

# Step 3: Define the objective function
@objective(waste_model, Min, 
sum(c_W[i,j]*W[i,j] for i in I, j in J) + # W
sum(c_R[k,j]*R[k,j] for k in K, j in J) + # R
sum(fixed_costs[j]*Y[j] for j in J)
)

# Step 4: Define the constraints

# city mass balance constraints
@constraint(waste_model, city1, W[1,1] + W[1,2] + W[1,3] == 100) 
@constraint(waste_model, city2, W[2,1] + W[2,2] + W[2,3] == 90) 
@constraint(waste_model, city3, W[3,1] + W[3,2] + W[3,3] == 120) 

# residual mass balance constaints
@constraint(waste_model, MRFresidual, R[2,3]+R[2,1] == 0.62(W[1,2] + W[2,2] + W[3,2]))
@constraint(waste_model, WTEresidual, R[3,1] == 0.16(W[1,3] + W[2,3] + W[3,3] + R[2,3]))

# disposal limit constraints
@constraint(waste_model, WTEwaste, W[1,3] + W[2,3] + W[3,3] + R[2,3] <= 210)
@constraint(waste_model, MRFwaste, W[1,2] + W[2,2] + W[3,2] + R[2,3] <= 350)
@constraint(waste_model, Lfwaste, W[1,1] + W[2,1] + W[3,1] + R[2,1] + R[3,1] <= 200)

# commitment 
M = 1000 # define M for Big-M reformulation 
@constraint(waste_model, LFcommit, Y[1] == 1) # landfill always on 
@constraint(waste_model, MRFcommit,  M*Y[2] >= W[1,2] + W[2,2] + W[3,2]) 
@constraint(waste_model, WTEcommit,  M*Y[3] >= W[1,3] + W[2,3] + W[3,3] + R[2,3])

print(waste_model)

Min 57.5 W[1,1] + 67.2 W[1,2] + 82.5 W[1,3] + 72.5 W[2,1] + 59.7 W[2,2] + 75 W[2,3] + 69.5 W[3,1] + 89.7 W[3,2] + 90 W[3,3] + 98 R[1,2] + 77 R[2,3] + 82.5 R[3,3] + 2000 Y[1] + 1500 Y[2] + 2500 Y[3]
Subject to
 city1 : W[1,1] + W[1,2] + W[1,3] == 100
 city2 : W[2,1] + W[2,2] + W[2,3] == 90
 city3 : W[3,1] + W[3,2] + W[3,3] == 120
 MRFresidual : -0.62 W[1,2] - 0.62 W[2,2] - 0.62 W[3,2] + R[2,1] + R[2,3] == 0
 WTEresidual : -0.16 W[1,3] - 0.16 W[2,3] - 0.16 W[3,3] + R[3,1] - 0.16 R[2,3] == 0
 LFcommit : Y[1] == 1
 MRFcommit : -W[1,2] - W[2,2] - W[3,2] + 1000 Y[2] >= 0
 WTEcommit : -W[1,3] - W[2,3] - W[3,3] - R[2,3] + 1000 Y[3] >= 0
 WTEwaste : W[1,3] + W[2,3] + W[3,3] + R[2,3] <= 210
 MRFwaste : W[1,2] + W[2,2] + W[3,2] + R[2,3] <= 350
 Lfwaste : W[1,1] + W[2,1] + W[3,1] + R[2,1] + R[3,1] <= 200
 W[1,1] >= 0
 W[2,1] >= 0
 W[3,1] >= 0
 W[1,2] >= 0
 W[2,2] >= 0
 W[3,2] >= 0
 W[1,3] >= 0
 W[2,3] >= 0
 W[3,3] >= 0
 R[1,1] >= 0
 R[2,1] >= 0
 R[3,1] >= 0
 R[1,2] >= 0
 R[2,2] >= 0
 R[3,2] >= 0
 

#### Problem 1.6 (2 points)

Find the optimal solution. Report the optimal objective value.

**Response:** The optimal objective value is: **$25,942.37**.

Moreover, we have: 

$Y_{1} = 1$, $Y_{2} = 1$, $Y_{3} = 0$

$W_{1,2} = 100$ Mg/day, $W_{2,2} = 90$ Mg/day, $W_{3,1} = 20.5$ Mg/day, $W_{3,2} = 99.5$ Mg/day

$R_{12,1} = 179.5$ Mg/day

In [41]:
# Solve the MILP
optimize!(waste_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 14 cols, 41 nonzeros
9 rows, 13 cols, 39 nonzeros

Solving MIP model with:
   9 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   39 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   2000            inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   2000            27312.880952      92.68%        0      0      0         0     0.0s
         0       0         0   0.00%   24197.578758    27312.880952      11.41%        0      0      0         6     0.0s
 S       0       0         0   0.00%   24197.578758    26179.52381        7.57%        7      1      0         6     0.0s
 H    

In [42]:
# Check the status of the optimization
status = termination_status(waste_model)
if status == MOI.OPTIMAL
    # Get the optimal solution values
    optimal_W = value.(W)
    optimal_R = value.(R)
    optimal_Y = value.(Y)

    # Print the results
    println("Optimal Waste Transport:\n", optimal_W)
    println("Optimal Residual Waste Transport:\n", optimal_R)
    println("Facility Operational Status:\n", optimal_Y)
else
    println("No optimal solution found. Status: ", status)
end

Optimal Waste Transport:
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:3
    Dimension 2, 1:3
And data, a 3×3 Matrix{Float64}:
 -0.0                100.0               0.0
  0.0                 90.0              -0.0
 20.526315789473703   99.4736842105263  -0.0
Optimal Residual Waste Transport:
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:3
    Dimension 2, 1:3
And data, a 3×3 Matrix{Float64}:
   0.0              0.0  0.0
 179.4736842105263  0.0  0.0
   0.0              0.0  0.0
Facility Operational Status:
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
 1.0
 1.0
 0.0


#### Problem 1.7 (5 points)

Draw a diagram showing the flows of waste between the cities and the
facilities. Which facilities (if any) will not be used? Does this
solution make sense?

**Response:** We should operate the Landfill (LF) and the Materials-Recycling Facility (MRF), based on my results from problem 1.6 (The resulting Y vector is [1,1,0]). So, we will *not* be using the Waste-to-Energy facility. My diagram of the system is shown below: 

### Problem 2 (18 points)

It is projected that in the near future the state will introduce a
carbon tax that will increase the cost for transportation and for
disposal by incineration. It is estimated that the additional costs will
be:

-   tipping fee for the WTE facility will increase to\$75/Mg; and
-   transportation costs will increase to \$2/Mg-km.

In this context, the cities are considering adding another landfill and
want to know if this would be cost-effective compared to using the
current facilities with the carbon tax. This landfill would have a
maximum capacity of 100 Mg/day and would be located with the following
distances from the existing sites (excluding LF1):

| **City/Facility** | **Distance to LF2 (km)** |
|:-----------------:|:------------------------:|
|         1         |            45            |
|         2         |            35            |
|         3         |            15            |
|        MRF        |            35            |
|        WTE        |            50            |

The fixed cost of operating this facility would be the same as the first
landfill, but the tipping cost would be increased to \$60/Mg-day.

#### Problem 2.1 (5 points)

What changes are needed to your optimization program from Problem 1 for
this decision problem? Formulate any different variables, objectives,
and/or constraints.

#### Problem 2.2 (3 points)

Implement the new optimization problem in `JuMP`.

#### Problem 2.3 (5 points)

Find the optimal solution and report the optimal objective value.
Provide a diagram showing the new waste flows.

#### Problem 2.4 (5 points)

Would you recommend that the cities build the new landfill? Why or why
not? Your answer should be based on your analysis but can draw on other
considerations as appropriate or desired.

## References

List any external references consulted, including classmates.